# Ejercicio adicional de fin de semana: semana 2

Ahora usa todo lo que aprendiste en la semana 2 para construir un prototipo completo para la pregunta/respuesta técnica que creaste en el ejercicio de la semana 1.

Esto debería incluir una interfaz de usuario de Gradio, transmisión, uso del mensaje del sistema para agregar experiencia y la capacidad de cambiar entre modelos. ¡Puntos extra si puedes demostrar el uso de una herramienta!

Si te sientes audaz, ve si puedes agregar una entrada de audio para poder hablarle y hacer que responda con audio. ChatGPT o Claude pueden ayudarte, o envíame un correo electrónico si tienes preguntas.

Pronto publicaré una solución completa aquí, a menos que alguien se me adelante...

Hay tantas aplicaciones comerciales para esto, desde un tutor de idiomas hasta una solución de incorporación de empresas, pasando por una IA complementaria para un curso (¡como este!). No puedo esperar a ver tus resultados.

Objetivos:

* Añade otra herramienta para hacer una reserva:
    * Imprimir mensaje por salida
    * Guardar en un fichero con día, lugar y precio de la reserva (en fichero csv o pdf).
* Agrega un Agente que traduzca todas las respuestas a otro idioma:
    * Muestra la traducción en el lado derecho:
    * Utilizan un modelo Frontier diferente (Claude)
    * Añade la selección del idioma.
* Añade un agente que pueda escuchar audio:
    * Convierte el audio a texto como entrada para el LLM.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr



In [2]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key sin configurar")

MODEL = "gpt-4o-mini"
openai = OpenAI()



OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = "Eres un asistente útil para una aerolínea llamada FlightAI. "
system_message += "Da respuestas breves y corteses, de no más de una oración. "
system_message += "Se siempre preciso. Si no sabes la respuesta, dilo."

# Se añaden nuevas instrucciones al sistema para que use las herramientas
system_message += "Cuando el cliente pregunte por precios, usa get_ticket_price. "
system_message += "Cuando el cliente confirme una reserva, usa save_booking con el destino y precio."

In [4]:
# Comencemos creando una función útil.

ticket_prices = {
    "londres": "$799",
    "parís": "$899",
    "tokyo": "$1400",
    "berlín": "$499"
}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [34]:
get_ticket_price("Londres")

Tool get_ticket_price called for Londres


'$799'

In [5]:
# Hay una estructura de diccionario particular que se requiere para describir nuestra función:

price_function = {
    "name": "get_ticket_price",
    "description": "Obtén el precio de un billete de ida y vuelta a la ciudad de destino. Llámalo siempre que necesites saber el precio del billete, por ejemplo, cuando un cliente pregunte '¿Cuánto cuesta un billete a esta ciudad?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "La ciudad a la que el cliente desea viajar",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# 1. Se añade otra herramienta: save_booking
* Guarda en un fichero csv el día, lugar y precio de la reserva

In [6]:
import csv
from datetime import datetime

# Función para guardar la reserva en un archivo CSV
def save_booking(destination_city, price):
    """Guarda la reserva en un archivo CSV con fecha, lugar y precio"""
    print(f"Guardando reserva para {destination_city} por {price}")

    # Obtener la fecha actual
    booking_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # Nombre del archivo CSV
    csv_file = "reservas.csv"

    # Verificar si el archivo existe para saber si necesitamos escribir el encabezado
    try:
        with open(csv_file, 'r') as f:
            file_exists = True
    except FileNotFoundError:
        file_exists = False

    # Escribir en el archivo CSV
    with open(csv_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        # Escribir encabezado si el archivo es nuevo
        if not file_exists:
            writer.writerow(['Fecha', 'Destino', 'Precio'])
        # Escribir la reserva
        writer.writerow([booking_date, destination_city, price])

    return f"Reserva guardada exitosamente para {destination_city} el {booking_date}"

In [7]:
# Estructura de diccionario para describir la función de guardar reserva

booking_function = {
    "name": "save_booking",
    "description": "Guarda una reserva de vuelo en un archivo CSV con la fecha actual, ciudad de destino y precio. Llama a esta función cuando el cliente confirme que quiere hacer una reserva.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "La ciudad de destino de la reserva",
            },
            "price": {
                "type": "string",
                "description": "El precio del billete",
            },
        },
        "required": ["destination_city", "price"],
        "additionalProperties": False
    }
}

In [8]:
# Y esto está incluido en una lista de herramientas:

tools = [
    {
        "type": "function",
        "function": price_function
    },
    {
        "type": "function",
        "function": booking_function
    }
]

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages
        )

    return response.choices[0].message.content

In [10]:
# Se modifica la función handle_tool_call para que maneje
# las llamadas a herramientas del LLM

def handle_tool_call(message):
    """Maneja las llamadas a herramientas del LLM"""
    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    # Determinar qué función llamar según el nombre
    if function_name == "get_ticket_price":
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        result = {"destination_city": city, "price": price}

    elif function_name == "save_booking":
        city = arguments.get('destination_city')
        price = arguments.get('price')
        booking_result = save_booking(city, price)
        result = {"message": booking_result, "destination_city": city, "price": price}

    else:
        result = {"error": f"Función desconocida: {function_name}"}

    # Crear la respuesta para el LLM
    response = {
        "role": "tool",
        "content": json.dumps(result),
        "tool_call_id": tool_call.id
    }

    return response, arguments.get('destination_city', 'Unknown')

In [41]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Londres
Guardando reserva para Londres por $799


# 2. Agente que traduce las respuestas a otro idioma
* Muestra la traducción en el lado derecho.
* Utilizan un modelo Frontier diferente (Claude).
* Añade la selección del idioma.

In [11]:
import anthropic

load_dotenv()

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')  # Añade esto a tu .env
if anthropic_api_key:
    print(f"Anthropic API Key configurada")
    anthropic_client = anthropic.Anthropic(api_key=anthropic_api_key)
else:
    print("Anthropic API Key no configurada")
    anthropic_client = None

Anthropic API Key configurada


In [12]:
def translate_text(text, target_language):
    """
    Traduce el texto al idioma objetivo

    Args:
        text: Texto a traducir
        target_language: Idioma destino (español, inglés, francés, etc.)
    """
    if not text or target_language == "Original (sin traducción)":
        return ""

    translation_prompt = f"Traduce el siguiente texto a {target_language}. Solo devuelve la traducción, sin explicaciones:\n\n{text}"

    try:
        response = anthropic_client.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=1024,
            messages=[{"role": "user", "content": translation_prompt}]
        )
        return response.content[0].text

    except Exception as e:
        return f"Error en traducción: {str(e)}"

In [13]:
def chat(message, history, target_language="Original (sin traducción)"):
    """
    Función de chat que incluye traducción

    Args:
        message: Mensaje del usuario
        history: Historial de conversación
        target_language: Idioma para traducir
    """
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response_data, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response_data)
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages
        )
    reply = response.choices[0].message.content

    # Traducir la respuesta
    translation = translate_text(reply, target_language)

    return reply, translation

In [63]:
with gr.Blocks() as ui:
    gr.Markdown("# FlightAI - Asistente de Aerolínea")

    with gr.Row():
        # Columna izquierda: Chatbot original
        with gr.Column(scale=1):
            gr.Markdown("### Conversación Original")
            chatbot = gr.Chatbot(
                height=500,
                label="Asistente FlightAI"
            )

        # Columna derecha: Traducción
        with gr.Column(scale=1):
            gr.Markdown("### Traducción")
            translation_box = gr.Textbox(
                label="Traducción de la última respuesta",
                lines=15,
                interactive=False,
                placeholder="La traducción aparecerá aquí..."
            )

    with gr.Row():
        # Selector de idioma
        language_selector = gr.Dropdown(
            choices=[
                "Original (sin traducción)",
                "Español",
                "Inglés",
                "Francés",
                "Alemán",
                "Italiano",
                "Portugués",
                "Japonés",
                "Chino"
            ],
            value="Original (sin traducción)",
            label="Idioma de traducción"
        )

    with gr.Row():
        entry = gr.Textbox(
            label="Escribe tu mensaje:",
            placeholder="Pregunta sobre vuelos, precios o haz una reserva..."
        )

    def do_entry(message, history, language):
        """Procesa la entrada del usuario"""
        history += [{"role": "user", "content": message}]
        return "", history

    def process_chat(history, language):
        """Procesa el chat y genera traducción"""
        if not history or history[-1]["role"] != "user":
            return history, ""

        user_message = history[-1]["content"]
        reply, translation = chat(user_message, history[:-1], language)

        history += [{"role": "assistant", "content": reply}]
        return history, translation

    # Eventos
    entry.submit(
        do_entry,
        inputs=[entry, chatbot, language_selector],
        outputs=[entry, chatbot]
    ).then(
        process_chat,
        inputs=[chatbot, language_selector],
        outputs=[chatbot, translation_box]
    )

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


Opening in existing browser session.
Tool get_ticket_price called for Londres
Guardando reserva para Londres por $799


# 3. Agente que puede escuchar audio
* Convierte el audio a texto como entrada para el LLM.

In [29]:
# import gradio as gr
# def transcribe_audio(audio_path):
#     """Transcribe audio a texto usando Whisper de OpenAI"""
#     print(f"Audio path recibido: {audio_path}")

#     if audio_path is None:
#         print("No hay audio")
#         return ""

#     try:
#         with open(audio_path, "rb") as audio_file:
#             print("Enviando a Whisper...")
#             transcript = openai.audio.transcriptions.create(
#                 model="whisper-1",
#                 file=audio_file,
#                 language="es",  # Forzar español
#                 prompt="Esta es una conversación sobre vuelos y reservas de aerolínea."
#             )
#         print(f"Transcripcion: {transcript.text}")
#         return transcript.text
#     except Exception as e:
#         print(f"Error: {e}")
#         return f"Error: {str(e)}"


def transcribe_audio(audio_path):
    """Transcribe audio a texto usando Whisper de OpenAI"""
    print(f"Audio path recibido: {audio_path}")

    if audio_path is None:
        print("No hay audio")
        return ""

    # Validar que el archivo existe y tiene contenido
    import os
    if not os.path.exists(audio_path):
        return "Error: Archivo de audio no encontrado"

    file_size = os.path.getsize(audio_path)
    print(f"Tamaño del archivo: {file_size} bytes")

    if file_size < 1000:  # Menos de 1KB probablemente es silencio
        return "Error: Audio demasiado corto o vacío"

    try:
        with open(audio_path, "rb") as audio_file:
            print("Enviando a Whisper...")
            transcript = openai.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
                language="es"
                # Sin prompt para evitar interferencias
            )

        result = transcript.text.strip()
        print(f"Transcripcion: {result}")

        # Filtrar alucinaciones conocidas de Whisper
        hallucination_patterns = [
            "subtítulos realizados",
            "amara.org",
            "thanks for watching",
            "subscribe",
            "suscríbete",
            "gracias por ver"
        ]

        for pattern in hallucination_patterns:
            if pattern.lower() in result.lower():
                print(f"Detectada alucinación de Whisper: {result}")
                return "No se pudo transcribir el audio. Por favor, habla más claro o más cerca del micrófono."

        if len(result) < 2:
            return "Audio no reconocido. Intenta hablar más claro."

        return result

    except Exception as e:
        print(f"Error: {e}")
        return f"Error: {str(e)}"


def process_voice(audio_path, history):
    print(f"=== INICIO PROCESO ===")
    print(f"Audio: {audio_path}")
    print(f"History: {len(history)} mensajes")

    if audio_path is None:
        return history, "Sin audio"

    user_message = transcribe_audio(audio_path)
    if not user_message or user_message.startswith("Error"):
        return history, user_message or "Error desconocido"

    history = history + [{"role": "user", "content": user_message}]

    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        msg = response.choices[0].message
        tool_resp, city = handle_tool_call(msg)
        messages.append(msg)
        messages.append(tool_resp)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    reply = response.choices[0].message.content
    history = history + [{"role": "assistant", "content": reply}]
    print(f"=== FIN PROCESO ===")

    return history, user_message


In [30]:
with gr.Blocks() as demo:
    gr.Markdown("# FlightAI")
    chatbot = gr.Chatbot(height=400)
    transcription = gr.Textbox(
        label="Transcripcion",
        interactive=False
    )
    audio = gr.Audio(sources=["microphone"], type="filepath")
    btn = gr.Button("Enviar")
    btn.click(
        process_voice,
        inputs=[audio, chatbot],
        outputs=[chatbot, transcription]
    )

# Abrir en navegador externo, no en iframe
demo.launch(
    inbrowser=True,
    share=False
)

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


Opening in existing browser session.
